# <center>**Podstawy Sztucznej Inteligencji**</center>
## <center>Projekt 2</center>
### <center> Drzewa decyzyjne oraz sieci neuronowe przy predykcji spalonych obszarów lasu</center>



<center>Adrian Nadratowski</center>
<center>Marcin Skrzypkowski</center>

Celem projektu jest stworzenie modeli przewidujących spalony obszar lasu przy wykorzystaniu udostępnionych nam zestawów danych, pobranych z __Machine Learning Repository__ (https://archive.ics.uci.edu/ml/datasets/Forest+Fires). Naszym zadaniem było przeanalizować powyższe dane, a następnie zaprojektować, nauczyć i zwalidować dwa regresory: model sieci neuronowej oraz drzewa decyzyjnego. Sprawdzenie miało odbyć się z pomocą _k_-krotnej walidacji krzyżowej.

Biblioteki wykorzystane do projektu:
  * pandas
  * tensorflow
  * scikit-learn
  * matplotlib

# %TODO:
### do uzupełnienia:
  * kluczowe decyzje projektowe
  * opis struktury programu
  * instrukcja użytkownika (+opis interfejsu)
  * wkład poszczególnych autorów
  * analiza zbioru danych (również odnośnie brakujących wartości)
  

### Opis struktury programu

Projekt podzielony został na __X__ części. W pierwszej następuje analiza danych oraz sprawdzenie czy nie brajkuje jakichś wartości, które mogłyby wpłynąć na jakość uczenia. Następnie przedstawiona została implementacja drzew decyzyjnych oraz walidacja kilku modeli o różnych parametrach. Kolejnym punktem jest sieć neuronowa oraz jej walidacja. Na sam koniec obydwa sposoby modelowania zostały ze sobą porównane. 

### Instrukcja obsługi użytkownika

  * po uruchomieniu niniejszego projektu należy upewnić się, czy zainstalowane są wszystkie wymagane biblioteki wymienione we wprowadzeniu
  * w folderze _../data_ powinien znajdować się zbiór danych, na którym operować będzie poniższy skrypt
  * projekt składa się z __4__ głównych części: 
    1. import i walidacja danych
    2. model drzewa decyzyjnego
    3. model sieci neuronowej
    4. podsumowanie
    i w tej kolejności należy uruchamiać odpowiednie skrypty
  * każda część projektu uruchamiana jest kolejno od najwyższych do najniższych komórek
  * każda komórka jest szczegółowo opisana z wyjaśnieniem procesu w niej zaimplementowanego

### Analiza danych
Poniżej następuje import bibliotek niezbędnych do reprezentacji danych, ich import i reprezentacja

In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpt
import math
%matplotlib inline

In [8]:
data = pd.read_csv('../data/forestfires.csv')
data

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
5,8,6,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,0.00
6,8,6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,0.00
7,8,6,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,0.00
8,8,6,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,0.00
9,7,5,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,0.00


### Oznaczenia poszczególnych kolumn :
  * __X__ - współrzędna osi rzędnych na mapie zamieszczonej poniżej $[1:9]$
  * __Y__ - współrzędna osi odciętych na mapie zamieszczonej poniżej $[2:9]$
  * __month__ - miesiąc wystąpienia pożaru $[jan:dec]$
  * __day__ - dzień wystąpienia pożaru $[mon:sun]$
  * __FFMC__ - Fine Fuel Moisture Code - wartość wilgotności obliczona uwzględniając temperaturę, względną wilgotność powietrza, wiatr i opady deszczu $[{18.7}:{96.2}]$
  * __DMC__ - Duff Moisture Code - współczynnik średniej wilgotności warstw organicznych do średniej głębokości $[{1.1}:{291.3}]$
  * __DC__ - Drought Code - współczynnik wilgotności głębszych warstw organicznych (przydatny przy badaniu obiawów suszy) $[{7.9}:{860.6}]$
  * __ISI__ - Initial Spread Index - współczynnik oczekiwanego rozprzestrzeniania pożaru - łączy wpływy wiatru oraz FFMC $[{0.0}:{56.1}]$
  * __temp__ - temperatura w $C^o$ $[{2.2}:{33.3}]$
  * __RH__ - Relative Hiumidity - wiglotność względna w % $[{15.0}:{100}]$
  * __wind__ - wiatr w km/h $[{0.4}:{40}]$
  * __rain__ - opady deszczu w $\frac{mm}{m^2}$ $[{0.0}:{6.4}]$
  * __area__ - spalona powierzchnia lasu w $ha$ $[{0.0}:{1090.84}]$
  

![img](https://www.researchgate.net/profile/Paulo_Cortez3/publication/238767143/figure/fig1/AS:298804772392991@1448252017812/The-map-of-the-Montesinho-natural-park.png)

<em>Materiał bazowy dostępny na stronie:[http://www.dsi.uminho.pt/~pcortez/fires.pdf](http://www.dsi.uminho.pt/~pcortez/fires.pdf)
    
    

</em>